# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [114]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [115]:
weather_df = pd.read_csv('city_weather_data.csv')
weather_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,ABNUB,EG,27.110222,30.782707,60.80,63.0,0.0,5.75,25/04/2021
1,CAPE TOWN,ZA,-49.105595,4.582316,59.00,94.0,0.0,4.61,25/04/2021
2,LAWRENCE,US,39.733156,-85.922722,51.01,76.0,1.0,5.95,25/04/2021
3,ATBASAR,KZ,51.945282,69.158641,46.96,36.0,84.0,18.75,25/04/2021
4,MATAURA,PF,-88.095666,-144.270442,48.00,75.0,34.0,10.00,25/04/2021


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [116]:
gmaps.configure(api_key=g_key)


In [117]:
# Store latitude and longitude in locations
city_locations = weather_df[["Lat", "Lng"]]

# Set Humidity as Weight
humidity = weather_df["Humidity"].astype(float)

In [118]:
# Map Size
figure_layout = {'width': '500px', 'margin': '0 auto 0 auto'}
gmaps.figure()
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(city_locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [119]:
# Narrowdown based on preffered weather

temp = weather_df[(weather_df['Max Temp'] > 70) & (weather_df['Max Temp'] < 80)]
# Narrowdown wind speed
speed = temp[temp['Wind Speed'] < 10]

# Narrowdown Cloudiness
cloud = speed[speed['Cloudiness'] == 0]
cloud

# Narrowdown humidity
dest_df = cloud[cloud['Humidity'] < 80]
dest_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
180,SAINT-LOUIS,SN,17.210056,-17.822924,71.60,64.0,0.0,4.61,25/04/2021
381,ATAR,MR,21.296317,-12.871419,74.70,21.0,0.0,8.19,25/04/2021
434,NARRABRI,AU,-30.032765,148.236274,73.99,30.0,0.0,8.05,25/04/2021
447,PORT MACQUARIE,AU,-32.334031,159.935068,75.00,57.0,0.0,8.23,25/04/2021
490,VALENTIN GOMEZ FARIAS,MX,27.364481,-106.469258,71.01,16.0,0.0,1.61,25/04/2021
520,DHARCHULA,IN,32.287539,82.128618,77.79,19.0,0.0,3.42,25/04/2021


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [120]:
# Create new df to convert actual coordinate of the cities. 
hotel_df = dest_df[['City', 'Country']]
hotel_df = hotel_df.reset_index(drop = True)

# Generate empty column for new lat and long and hotel
hotel_df['Lat'] = ""
hotel_df['Lng'] = ""
hotel_df['Hotel Name'] =""
hotel_df
for index, row in hotel_df.iterrows():
    city = row['City']
    country = row['Country']

    target_city = (f'{city}, {country}')
    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&key={1}').format(target_city, g_key)

    geo_data = requests.get(target_url).json()
    print(f'Processing Record {index+1} of {len(hotel_df)} | {city}')

    hotel_df.at[index, 'Lat'] = geo_data['results'][0]['geometry']['location']['lat']
    hotel_df.at[index, 'Lng'] = geo_data['results'][0]['geometry']['location']['lng']


Processing Record 1 of 6 | SAINT-LOUIS
Processing Record 2 of 6 | ATAR
Processing Record 3 of 6 | NARRABRI
Processing Record 4 of 6 | PORT MACQUARIE
Processing Record 5 of 6 | VALENTIN GOMEZ FARIAS
Processing Record 6 of 6 | DHARCHULA


In [121]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,SAINT-LOUIS,SN,16.0326,-16.4818,
1,ATAR,MR,20.5073,-13.053,
2,NARRABRI,AU,-30.3324,149.781,
3,PORT MACQUARIE,AU,-31.4333,152.9,
4,VALENTIN GOMEZ FARIAS,MX,29.3613,-107.734,
5,DHARCHULA,IN,29.8473,80.5369,


In [123]:
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    lat=row['Lat']
    lng=row['Lng']
    params = {
        "location": f"{lat},{lng}",  
        # "rankby": "distance",
        "radius": "5000",
        "type": "lodging",
        "key": g_key,
        "rating": 4,
        "max price": 3,
    }

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    
    try:
        hotel_df.at[index, 'Hotel Name'] = response['results'][0]['name']
        print(response['results'][0]['name'])
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


Retrieving Results for Index 0: SAINT-LOUIS.
Auberge de Jeunesse du Sud
------------
Retrieving Results for Index 1: ATAR.
Oumou Elghoura
------------
Retrieving Results for Index 2: NARRABRI.
Crossroads Hotel
------------
Retrieving Results for Index 3: PORT MACQUARIE.
Rydges Port Macquarie
------------
Retrieving Results for Index 4: VALENTIN GOMEZ FARIAS.
Hotel Victoria
------------
Retrieving Results for Index 5: DHARCHULA.
Mahara Hotel
------------


In [124]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,SAINT-LOUIS,SN,16.0326,-16.4818,Auberge de Jeunesse du Sud
1,ATAR,MR,20.5073,-13.053,Oumou Elghoura
2,NARRABRI,AU,-30.3324,149.781,Crossroads Hotel
3,PORT MACQUARIE,AU,-31.4333,152.9,Rydges Port Macquarie
4,VALENTIN GOMEZ FARIAS,MX,29.3613,-107.734,Hotel Victoria
5,DHARCHULA,IN,29.8473,80.5369,Mahara Hotel


In [125]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [127]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))